In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import time

import CustomHyperModel
import CustomWindowGenerator
import EnergyPricesLibrary as Ep

from kerastuner.tuners import BayesianOptimization

%load_ext autoreload
%autoreload 2

## Arquitectura con Repeat Vector

In [4]:
n_steps_in = 24
len_input_features = 84 
model = tf.keras.Sequential()

model.add(
    tf.keras.layers.LSTM(
        input_shape=(n_steps_in,len_input_features),
        units=512,
        activation='tanh',
        kernel_regularizer=tf.keras.regularizers.L1(l1=0),
        dropout=0,
        return_sequences=True
    )
)

model.add(
    tf.keras.layers.LSTM(   
        units=512,
        activation='tanh',
        kernel_regularizer=tf.keras.regularizers.L1(l1=0),
        dropout=0,
        return_sequences=False
    )
)

model.add(tf.keras.layers.RepeatVector(24))
        
model.add(
    tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(
            units=120,
            activation='relu'
        )
    )
)

model.add(tf.keras.layers.Dropout(rate=0))


model.add(
    tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(units=1,activation=None)
    )
)

model.compile(
    optimizer=tf.optimizers.Adam(0.0001),
    loss=tf.losses.MeanSquaredError(),
    metrics=[
        tf.metrics.MeanAbsoluteError(),
        tf.keras.metrics.MeanAbsolutePercentageError()],
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 24, 512)           1222656   
_________________________________________________________________
lstm_5 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 24, 512)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 24, 120)           61560     
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 120)           0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 24, 1)             121       
Total params: 3,383,537
Trainable params: 3,383,537
Non-trainable params: 0
____________________________________________

## Arquitectura con Flatten

In [7]:
n_steps_in = 48
len_input_features = 84 
model = tf.keras.Sequential()

model.add(
    tf.keras.layers.LSTM(
        input_shape=(n_steps_in,len_input_features),
        units=512,
        activation='tanh',
        kernel_regularizer=tf.keras.regularizers.L1(l1=0),
        dropout=0,
        return_sequences=True
    )
)

model.add(
    tf.keras.layers.LSTM(   
        units=512,
        activation='tanh',
        kernel_regularizer=tf.keras.regularizers.L1(l1=0),
        dropout=0,
        return_sequences=True
    )
)
        
model.add(
    tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(
            units=120,
            activation='relu'
        )
    )
)

model.add(tf.keras.layers.Dropout(rate=0))

model.add(
    tf.keras.layers.Flatten()
)

model.add(
tf.keras.layers.Dense(units=24,activation=None)
)

model.compile(
    optimizer=tf.optimizers.Adam(0.0001),
    loss=tf.losses.MeanSquaredError(),
    metrics=[
        tf.metrics.MeanAbsoluteError(),
        tf.keras.metrics.MeanAbsolutePercentageError()],
)

model.summary()

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 48, 512)           1222656   
_________________________________________________________________
lstm_11 (LSTM)               (None, 48, 512)           2099200   
_________________________________________________________________
time_distributed_9 (TimeDist (None, 48, 120)           61560     
_________________________________________________________________
dropout_5 (Dropout)          (None, 48, 120)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5760)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 24)                138264    
Total params: 3,521,680
Trainable params: 3,521,680
Non-trainable params: 0
____________________________________________

In [20]:
n_steps_in = 48
len_input_features = 84 
model = tf.keras.Sequential()

model.add(
    tf.keras.layers.LSTM(
        input_shape=(n_steps_in,len_input_features),
        units=512,
        activation='tanh',
        kernel_regularizer=tf.keras.regularizers.L1(l1=0),
        dropout=0,
        return_sequences=True
    )
)

model.add(
    tf.keras.layers.LSTM(
        input_shape=(n_steps_in,len_input_features),
        units=512,
        activation='tanh',
        kernel_regularizer=tf.keras.regularizers.L1(l1=0),
        dropout=0,
        return_sequences=True
    )
)
        
model.add(
            tf.keras.layers.Flatten()
        )

model.add(
tf.keras.layers.Dense(units=24,activation=None)
)

model.compile(
    optimizer=tf.optimizers.Adam(0.0001),
    loss=tf.losses.MeanSquaredError(),
    metrics=[
        tf.metrics.MeanAbsoluteError(),
        tf.keras.metrics.MeanAbsolutePercentageError()],
)

model.summary()

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 48, 512)           1222656   
_________________________________________________________________
lstm_33 (LSTM)               (None, 48, 512)           2099200   
_________________________________________________________________
flatten_5 (Flatten)          (None, 24576)             0         
_________________________________________________________________
dense_30 (Dense)             (None, 24)                589848    
Total params: 3,911,704
Trainable params: 3,911,704
Non-trainable params: 0
_________________________________________________________________
